In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (25, 10)}, font_scale=1.5)

df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')

# df = pd.read_csv('ECH_2019.csv')

In [3]:
df.sample(2)



,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof,barrio,estado_civil,hijos,hijos en hogar,hijos en otro hogar,hijos en el extranjero,leer_escribir,suma_hijos
102687,Hombre,42,Blanca,Jefe/a de hogar,Ocupados,19000,No,"Producción, transmisión y distribución de ener...",Sí,"Sí, asistió","Sí, asistió",No asistió,No asistió,0,0,0,0,0,Soltero/a,0,0,0,0,Sí,0
48232,Mujer,76,Blanca,Jefe/a de hogar,Inactivo: jubilado,0,0,.,Sí,No asistió,"Sí, asistió",No asistió,"Sí, asistió",0,0,0,0,26,Separado/a de unión libre anterior,Sí,0,2,0,Sí,2


In [4]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad',
                              'sueldo',
                              'hijos',
                              'hijos en hogar',
                              'hijos en otro hogar',
                              'hijos en el extranjero',
                              'suma_hijos',
                              'barrio'])

df[mask] = df[mask].replace(no_data)



### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Seleccionamos personas con edad suficiente para trabajar

In [5]:
df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]

In [6]:
df = df[df['estado_laboral'].str.contains('Ocupados')]

# Categorizamos las edades y sueldos en rangos

In [7]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[
    1,
    5000,
    10000,
    30000,
    50000,
    100000,
    float('Inf'),
    ]).cat.codes

df['rango_edades'] = pd.cut(df['edad'], bins=[
    0,
    10,
    15,
    20,
    30,
    40,
    50,
    60,
    70,
    float('Inf'),
    ]).cat.codes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Eliminanos las columnas que no vamos a usar o que ya categorizamos

In [8]:
df = df[[
    'sexo', 
    'edad', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'sueldo', 
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'barrio',
    'estado_civil',
    'suma_hijos',
    'leer_escribir',
    'rango_sueldos',
    'rango_edades'
    ]]
    

In [9]:
df = df[~df['rango_sueldos'].isna()]

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [10]:
# Feautures
X = df.drop('rango_sueldos', axis=1).to_numpy()
y = df['rango_sueldos']

print('X shape', X.shape, '\nY shape', y.shape)

X shape (49036, 22) 
Y shape (49036,)


In [11]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)


In [12]:
prediction = X.toarray()
prediction = prediction[49000].reshape(1, -1)

### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [13]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 



In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(34325, 1867) (34325,)
(14711, 1867) (14711,)


# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(multi_class='multinomial', max_iter=20000)

logistic_reg.fit(X_train, y_train)



LogisticRegression(max_iter=20000, multi_class='multinomial')

### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [16]:
scores = cross_val_score(logistic_reg, X, y, cv=5)

scores



array([0.9891925 , 0.98755991, 0.98755991, 0.98847762, 0.98817171])

In [17]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.99
Score en test  0.99
Score en train  1.0


In [18]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)



array([[4542,    0,    0,    0,    0,    0,    0],
       [   0,  407,    2,   21,    0,    5,    0],
       [   0,    2,  798,   34,    1,    3,    0],
       [   0,    0,    0, 5684,    4,    2,    0],
       [   0,    0,    0,   48, 2196,    8,    3],
       [   0,    0,    0,   13,    6,  768,    1],
       [   0,    0,    0,    2,    3,   27,  131]])

### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [19]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



Aciertos sobre entrenamiento: 1.0
Aciertos sobre evaluación: 0.99


In [20]:
y_train

107625   -1
84723     3
61045     4
46237     2
81096    -1
         ..
24931     3
98341     2
84144     3
1898      2
34996    -1
Name: rango_sueldos, Length: 34325, dtype: int8

### APLICAMOS PREDICCIÓN SOBRE EL MODELO

In [21]:
print(logistic_reg.predict(prediction))

print(y_train[49000])



[3]
-1


Determinamos la probabilidad de la predicción

In [22]:
probabilidades_prediccion = logistic_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]



array([8.47220983e-05])

# REPETIMOS EL EJERCICIO CON EL MODELO DE RANDOM FOREST

In [23]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)



RandomForestClassifier()

In [24]:
scores = cross_val_score(random_forest, X, y, cv=5)

scores



array([0.94606444, 0.94401958, 0.94534516, 0.94616091, 0.94911798])

In [25]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.95
Score en test  0.99
Score en train  1.0


In [26]:
predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)



array([[4542,    0,    0,    0,    0,    0,    0],
       [   0,  407,    2,   21,    0,    5,    0],
       [   0,    2,  798,   34,    1,    3,    0],
       [   0,    0,    0, 5684,    4,    2,    0],
       [   0,    0,    0,   48, 2196,    8,    3],
       [   0,    0,    0,   13,    6,  768,    1],
       [   0,    0,    0,    2,    3,   27,  131]])

In [27]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



Aciertos sobre entrenamiento: 1.0
Aciertos sobre evaluación: 0.94


In [28]:
print(random_forest.predict(prediction))
print(y_train[49000])



[3]
-1


# REPETIMOS EL EJERCICIO CON EL MODELO DE K NEAREST NEIGHBOURS

In [29]:
from sklearn.neighbors import KNeighborsClassifier

knearest_model = KNeighborsClassifier(n_neighbors=4)

knearest_model.fit(X_train, y_train)



KNeighborsClassifier(n_neighbors=4)

In [30]:
scores = cross_val_score(knearest_model, X, y, cv=5)

scores           



array([0.6673124 , 0.65494035, 0.65840726, 0.6586112 , 0.65983481])

In [31]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(knearest_model.score(X_test, y_test),2))
print('Score en train ', round(knearest_model.score(X_train, y_train),2))



Promedio de cross_validation  0.66
Score en test  0.66
Score en train  0.78


In [32]:
predictions = knearest_model.predict(X_test)

confusion_matrix(y_test, predictions)



array([[4492,    2,    1,   28,   16,    3,    0],
       [ 158,   74,   55,  138,    9,    1,    0],
       [ 280,   39,  210,  274,   24,    9,    2],
       [1367,   45,  135, 3781,  306,   55,    1],
       [ 542,    7,   18,  722,  880,   79,    7],
       [ 230,    3,    3,  148,  187,  199,   18],
       [  60,    0,    1,   22,   20,   41,   19]])

In [33]:
#Prediccion en Train
y_train_pred = knearest_model.predict(X_train)

#Prediccion en Test
y_test_pred = knearest_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



Aciertos sobre entrenamiento: 0.78
Aciertos sobre evaluación: 0.66


In [34]:
print(knearest_model.predict(prediction))
print(y_train[49000])



[2]
-1


# El porcentaje alto de resultados puede signifcar que hay overfitting o bias. Tanto la base como los modelos necesitan customización o se puede hacer un sampling para comparar los resultados. El modelo de KNN podría considerarse el mejor condidato.